In [1]:
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
from pyspark.ml.feature import StringIndexer
import pyspark.sql.functions as func
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.mllib.evaluation import RankingMetrics
from pyspark import SparkContext
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np
import sys
import getpass

In [2]:
spark = SparkSession.builder.config("spark.driver.memory", "16G").config('spark.executor.memory', '16G').appName('sampler').getOrCreate()
sc = spark.sparkContext

In [3]:
trainSample = spark.read.parquet('train_sample1.parquet')
testSample = spark.read.parquet('test_sample1.parquet')
valSample = spark.read.parquet('val_sample1.parquet')
    
#valSample.createOrReplaceTempView('valSample')
#trainSample.createOrReplaceTempView('trainSample')
#testSample.createOrReplaceTempView('testSample')

In [4]:
indexer_obj_1 = StringIndexer(inputCol="user_id", outputCol="user_id_numer").setHandleInvalid("keep")
indexer_model_1 = indexer_obj_1.fit(trainSample)
indexer_df_1 = indexer_model_1.transform(trainSample)

In [5]:
indexer_obj_2 = StringIndexer(inputCol="track_id", outputCol="track_id_numer").setHandleInvalid("keep")
indexer_model_2= indexer_obj_2.fit(indexer_df_1)
indexer_df_2 = indexer_model_2.transform(indexer_df_1)

In [6]:
train_df = indexer_df_2.drop('user_id')
train_df = train_df.drop('track_id')

In [ ]:
train_df = train_df.repartition(2000)

In [7]:
val_df_1 = indexer_model_1.transform(valSample)

In [8]:
val_df_2 = indexer_model_2.transform(val_df_1)

In [9]:
val_df = val_df_2.drop('user_id')
val_df = val_df.drop('track_id')
    
val_df = val_df.repartition(5000)

In [10]:
test_df_1 = indexer_model_1.transform(testSample)
test_df_2 = indexer_model_2.transform(test_df_1)

test_df = test_df_2.drop('user_id')
test_df = test_df.drop('track_id')

test_df = test_df.repartition(5000)

In [11]:
als = ALS(userCol="user_id_numer",itemCol="track_id_numer",ratingCol="count",
                         coldStartStrategy="drop",implicitPrefs=True,rank=int(20),regParam=float(0.1))

In [12]:
print("model trained")
best_model = als.fit(train_df)
print("fitted")

model trained
fitted


In [13]:
users = test_df.select(als.getUserCol()).distinct()
test_preds = best_model.recommendForUserSubset(users,5)
test_preds_explode = test_preds.select(test_preds.user_id_numer,func.explode(test_preds.recommendations.track_id_numer))
test_preds_flatten = test_preds_explode.groupby('user_id_numer').agg(func.collect_list('col').alias("col"))

In [14]:
test_true_df = test_df.groupby('user_id_numer').agg(func.collect_list('track_id_numer').alias("track_id_numer"))
test_true_flatten = test_true_df.repartition(5000)

In [15]:
rankingsRDD = (test_preds_flatten.join(test_true_flatten, 'user_id_numer').rdd.map(lambda row: (row[1], row[2])))

In [16]:
metrics = RankingMetrics(rankingsRDD)

print("Ranking Metrics called")
    
MAP = metrics.meanAveragePrecision
print(MAP)

Ranking Metrics called
0.0018038237738985873


In [17]:
rankingsRDD.collect()

[([153, 36, 7, 25, 34], [27020.0]),
 ([6, 29, 25, 2, 96], [604.0]),
 ([6, 10, 0, 14, 86], [106998.0]),
 ([22, 122, 67, 88, 56], [22814.0]),
 ([0, 3, 10, 25, 7], [13421.0]),
 ([4, 1, 52, 114, 10], [13290.0, 41908.0]),
 ([26, 30, 5, 114, 7], [2979.0]),
 ([23, 39, 19, 83, 29], [16435.0]),
 ([5, 13, 20, 72, 30], [624.0]),
 ([4, 30, 26, 28, 25], [11967.0]),
 ([104, 21, 10, 30, 6], [21268.0]),
 ([25, 0, 9, 7, 6], [52487.0]),
 ([15, 39, 4, 77, 27], [12.0]),
 ([31, 417, 3, 9, 22], [261.0]),
 ([39, 8, 47, 4, 51], [1811.0]),
 ([13, 10, 0, 41, 49], [106880.0, 106998.0]),
 ([0, 3, 14, 35, 827], [4037.0]),
 ([11, 39, 51, 88, 4], [5196.0]),
 ([13, 145, 201, 2, 15], [87.0]),
 ([2, 114, 217, 5, 30], [740.0]),
 ([14, 7, 54, 115, 24], [29618.0, 92300.0]),
 ([15, 82, 11, 2, 19], [3110.0]),
 ([103, 14, 75, 54, 36], [6481.0]),
 ([64, 4, 22, 6, 45], [28.0]),
 ([52, 1, 2, 10, 148], [46075.0]),
 ([36, 26, 7, 34, 19], [41968.0]),
 ([67, 79, 38, 2, 26], [106998.0]),
 ([83, 1, 23, 19, 198], [1972.0]),
 ([16, 896